**CONJUNTO INDIVIDUAL**

El conjunto individual nos permitr hallar la lista de canciones totales sin repeticiones, esto nos servira para poder buscar las frecuencias dentro del archivo **spotifys.npy**

In [1380]:
def conjuntoindividual(playlists):
    Listadecanciones = set()
    for i in playlists:
        Listadecanciones.update(i)
    Listadecanciones = list(frozenset(Listadecanciones))
    return Listadecanciones
#Itemset = list(frozenset(Listadecaciones))
#Itemset
#len(Itemset)

**get_frequent_itemset**

Este modulo nos permite generar las frecuencias de los datos y almacenar el valor de support de cada uno.

Los parametros son:

playlist: Es la lista de todas las playlist que existe en el archivo **spotifys.npy** es con el cual haremos la busquedaq de las frecuencias.
min_support: Es el minimo soporte que debe tener la cancion para ser seleccionado.
Listadecanciones: Es la lista de todas las canciones existente sin repeticiones la cual nos permitira buscar la frecuencia de estas mismas 

In [1381]:
def get_frequent_itemsets(playlists, min_support, Listadecanciones):
    Frecuencias = dict()
    Soporte = dict()
    for i in playlists:
        for j in Listadecanciones:
            prueba= set(j.split(", "))
            if (prueba).issubset(i):
                if not j in Frecuencias:
                    Frecuencias[j] = 1
                else:
                    Frecuencias[j] += 1
    total = len(playlists)
    Poda = []
    for l in Frecuencias:   
        sup = Frecuencias[l]/total
        if (sup >= min_support):
            Soporte[l] = sup
            Poda.append(l)
    return Poda, Soporte


**cambio**
Es el modulo que nos permite eliminar los datos de exceso dentro de los nombres de las canciones en este caso las doble comillas y los corchetes.

In [1382]:
def cambio(palabra):
    pala = ""
    for i in palabra:
        if (i != '{' and i != '}' and i != "'" and i != "'"):
            pala += i
        #else:
         #   pala.replace('"',"'")
            
    return pala

**combinaciones**
Es el modulo que se encarga de realizar las combinaciones de las canciones {1}, {1,2}, {1,2,3}...etc 

Los parametro:

Poda: Es la lista ya podada la cual nos ayudara a realizar las combinaciones entre ellas mismas.
tamañodebusqueda: Es el tamaño de las combinaciones que se realizara.

In [1383]:
def combinaciones(Poda, tamañodebusqueda):
    tamlista = len(Poda)
    combinacion= dict()
    for i in range(tamlista):
        for j in range(i+1, tamlista):
            Podak = set(Poda[i].split(", ")) | set(Poda[j].split(", "))
            if len(Podak) == tamañodebusqueda:
                Podak = cambio(str(Podak))
                if (Podak in combinacion) != True:
                    combinacion[Podak] = 1
    return list(combinacion)                         

Importamos las librerias necesarias en este caso la de **numpy**.

In [1384]:
import numpy as np

Cargamos los datos del archivo **"spotifys.npy"**

In [1385]:
Datos = np.load("spotifys.npy",allow_pickle=True)

Convertimos el archivo cargado en una lista y este lo almacenamos en una nueva varible **"Listadedatos"**

In [1386]:
Listadedatos = Datos.tolist()

Pasamos a hallar el total de canciones sin sus repeticiones para ellos utilizamos la funcion: 

 1.- **set()** la cual se encarga de eliminar los datos repetidos y a la vez nos devuelve los datos en orden de menor a mayor, en este caso siendo datos categoricos lo ordena de acuerdo al orden orden alfabetico y en caso existan valores especiales (", #, %, !, ...) estos van primero.

 2.- Utilizamos un **for** ya que este nos permite iterar en la variable de datos **Listadedatos** y asi obtenemos las canciones totales 

 3.- Guardamos los datos en la variable **playlists**

In [1387]:
playlists = [set(Listadedatos[i]) for i in Listadedatos ]
len(playlists)

10000

Llamamos a la funcion **conjuntoindividual** y guardamos los resultados en la variable **Listadecanciones** 

In [1388]:
Listadecanciones = conjuntoindividual(playlists)

Llamamos a la **funcion get_frequent_itemsets** en la cual pasamos los datos de **playlist**, el soporte que utilizaremos que sera de 0.01, y por ultimo pasamos los datos de **Listadecanciones** para obtener las canciones que cubren el soporte indicado.

In [1389]:
Poda, Soporte = get_frequent_itemsets(playlists, 0.01, Listadecanciones)
len(Poda)

841

Creamos un array donde guardaremos todos los **valores podados** ya que estos nos ayudaran a saber cual es la ultima iteracion(esto sera mejor explicado en la utilizacion de todos los modulos).

In [1390]:
Podas = [Poda]

Inicializamos una variable que nos permita recuperar la primera lista podada que este caso seria la de la posicion 0 ya que dentro de **Podas** solo existe **Poda** en dicha posicion.

**tamañodebusqueda = 0** 

In [1391]:
tamañodebusqueda=0

Pero teniendo el conocimiento que **tamañodebusqueda** nos permitira saber las veces que debe combinarse los datos en este caso Poda tiene los datos de manera individual lo cual ahora tocaria hallar las combinaciones de 2 en 2 para ello tendriamos que el valor de la variable **tamañodebusqueda** seria de 2.

Pero como anteriormente la habiamos iniciado en 0 ahora la cambiamos a 2 para poder simplificar el conteo de datos dentro de **Poda** y no tener que buscar la combinacion existente dentro de esta.

In [1392]:
tamañodebusqueda = 2

Pasamos a hallar las **podas** y **soportes** de todas las combinaciones, pero como esta cantidad de combinaciones no es conocida pasamos a utilizar un while en el cual haremos uso de la del tamaño de **Podas** ya que si este al actualizarse esta vacio no tendra un tamaño y sera igual a **0** siendo este el que determine la cantidad de veces que se utilice el while

1.- En la linea 2 lo que buscamos es recuperar todas las combinaciones totales, para esto usamos el modulo **combinaciones** al cual le pasamos el arreglo de todas las podas y recuperamos el primero el cual tiene todos los valores individuales, y como segundo parametro la variable **tamañodebusqueda** la cual definimos en 2 ya que este nos permitira generar las combinaciones de 2 en 2.

2.- En la linea 3 recuperamos el nuevo valor podado y el soporte del mismo.

3.- En la linea 4 agregamos el nuevo soporte a nuestro array de soportes ya que este nos servira para hallar el confidence y el lift

4.- En la linea 5 agregarmos la nueva poda al valor de podas.

5.- Y por ultimo incrementamos nuestra varable **tamañodebusqueda** en 1 ya que ahora pasaremos a hallar el siguiente grupo de combinaciones con este dato 

In [1393]:
while (len(Podas[tamañodebusqueda-2]) > 0):
    combi = combinaciones(Podas[tamañodebusqueda-2], tamañodebusqueda) 
    Poda, Sopor = get_frequent_itemsets(playlists, 0.01, combi)
    Soporte.update(Sopor)
    Podas.append(Poda)
    tamañodebusqueda += 1 


Mostramos todos los datos.

In [1543]:
Podas

[['Paper Planes',
  "Stacy's Mom",
  'Say My Name',
  'Party In The U.S.A.',
  'Toxic',
  'Where Is The Love?',
  'Crazy In Love',
  'Miss Independent',
  'Replay',
  "It Wasn't Me",
  'Dirty Little Secret',
  'Promiscuous',
  'Forever',
  'Whatcha Say',
  'All The Small Things',
  'Yeah!',
  'Mr. Brightside',
  'Closer',
  'See You Again',
  'Hey Ya! - Radio Mix / Club Mix',
  'Home',
  'Sympathy For The Devil',
  'Doses & Mimosas',
  'Lean On (feat. MØ & DJ Snake)',
  'Juicy',
  'Sweet Emotion',
  'Bohemian Rhapsody - Remastered 2011',
  'More Than a Feeling',
  'Snow (Hey Oh) - Hey Oh Album Version',
  "Don't Stop Believin'",
  'Do It Again',
  'Seven Nation Army',
  'No Type',
  'ABC',
  'You & Me - Flume Remix',
  "Jessie's Girl",
  'Big Poppa',
  'The Next Episode',
  'Ignition - Remix',
  'Billie Jean',
  'Man Of The Year',
  '1985',
  'Hypnotize - 2014 Remastered Version',
  'No Role Modelz',
  'Stronger',
  'Free Bird',
  'Changes',
  'Work REMIX',
  'Summertime',
  'Summertim

#convertimos todos los datos podados en 

In [1590]:
#recuperamos todos los datos finales podados
def conver(Podas):
    tamaño = len(Podas)
    listas=[]
    for i in Podas[tamaño-2]:
        listas.append(i.split(','))
    return listas
prueba = conver(Podas)    
prueba


[['Congratulations', ' XO TOUR Llif3', ' goosebumps'],
 ['Congratulations', ' goosebumps', ' HUMBLE.'],
 ['Congratulations', ' XO TOUR Llif3', ' HUMBLE.'],
 ['Congratulations', ' XO TOUR Llif3', ' Mask Off'],
 ['XO TOUR Llif3', ' HUMBLE.', ' goosebumps'],
 ['XO TOUR Llif3', ' Mask Off', ' goosebumps'],
 ['Congratulations', ' Bad and Boujee (feat. Lil Uzi Vert)', ' HUMBLE.'],
 ['Congratulations', ' Mask Off', ' HUMBLE.'],
 ['goosebumps', ' HUMBLE.', ' XO TOUR Llif3'],
 ['Mask Off', ' goosebumps', ' HUMBLE.'],
 ['XO TOUR Llif3', ' HUMBLE.', ' Slippery (feat. Gucci Mane)'],
 ['Mask Off', ' XO TOUR Llif3', ' HUMBLE.'],
 ['HUMBLE.', ' XO TOUR Llif3', ' Mask Off'],
 ['Mask Off', ' Bounce Back', ' HUMBLE.'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  ' Broccoli (feat. Lil Yachty)',
  ' Bounce Back'],
 ['Bad and Boujee (feat. Lil Uzi Vert)',
  ' Bounce Back',
  ' Broccoli (feat. Lil Yachty)'],
 ['HUMBLE.', ' Bounce Back', ' Mask Off'],
 ['HUMBLE.', ' goosebumps', ' Mask Off'],
 ['Bad and Bouje

**Metodo para generar las combinaciones**

In [ ]:
#Metodo para hallar las combinadas
def potencia(pValor):
    if len(pValor) == 0:
        return [[]]
    r = potencia(pValor[:-1])
    return r + [s + [pValor[-1]] for s in r]
def combinacionesv(A, pValor):
  valores = [s  for s in potencia(A) if len(s) == pValor]
  return valores

Metodo para hallar las combinaciones de las regalas

In [1592]:
#Para hallar las combinaciones de reglas
def arreglonuevo(Arreglo,SubArreglo):
  ArregloResultante=[]
  for i in Arreglo:
    if(not(i in SubArreglo)):
      ArregloResultante.append(i)
  return ArregloResultante

**Metodo para generar las reglas e imprimirlas**

In [1593]:
def Generar_Reglas(A):
  Reglas=[]
  for i in range(len(A)+1):
    Combinacion=combinacionesv(A, i+1)
    for j in range(len(Combinacion)):
      CombinacionRestante=combinacionesv(arreglonuevo(A,Combinacion[j]),(len(A)-(i+1)))
      #impresion de reglas
      print(Combinacion[j]," => ",CombinacionRestante[0])
      #agregar las reglas
      Reglas.append([Combinacion[j],CombinacionRestante[0]])
  return Reglas

#conjunto de reglas generadas
nuevo = []
for i in prueba:
  nuevo.append(Generar_Reglas(i))

['Congratulations']  =>  [' XO TOUR Llif3', ' goosebumps']
[' XO TOUR Llif3']  =>  ['Congratulations', ' goosebumps']
[' goosebumps']  =>  ['Congratulations', ' XO TOUR Llif3']
['Congratulations', ' XO TOUR Llif3']  =>  [' goosebumps']
['Congratulations', ' goosebumps']  =>  [' XO TOUR Llif3']
[' XO TOUR Llif3', ' goosebumps']  =>  ['Congratulations']
['Congratulations', ' XO TOUR Llif3', ' goosebumps']  =>  []
['Congratulations']  =>  [' goosebumps', ' HUMBLE.']
[' goosebumps']  =>  ['Congratulations', ' HUMBLE.']
[' HUMBLE.']  =>  ['Congratulations', ' goosebumps']
['Congratulations', ' goosebumps']  =>  [' HUMBLE.']
['Congratulations', ' HUMBLE.']  =>  [' goosebumps']
[' goosebumps', ' HUMBLE.']  =>  ['Congratulations']
['Congratulations', ' goosebumps', ' HUMBLE.']  =>  []
['Congratulations']  =>  [' XO TOUR Llif3', ' HUMBLE.']
[' XO TOUR Llif3']  =>  ['Congratulations', ' HUMBLE.']
[' HUMBLE.']  =>  ['Congratulations', ' XO TOUR Llif3']
['Congratulations', ' XO TOUR Llif3']  =>  [

**Modulo para eliminar los esapcios y buscar el soporte en el arreglo soporte**

In [ ]:
#elimna los espacios de mas para buscar en el soporte
def bus(union):
    buscar = ""
    contador = 0
    tam = len(union)
    for i in union:
        letras = 0
        palabra = ""
        for k in i:
            if (letras == 0):
                if (k==" "):
                    palabra = palabra + ""
                else: 
                    palabra = palabra + k
            else:
                palabra = palabra + k
            letras += 1
        if (contador < tam-1):
            buscar = buscar +palabra+", " 
        else:
            buscar = buscar+palabra
        contador += 1
        buscar.replace(" ","")
    return buscar

**Modulo para ver si cumple el lift y confidence**

In [1594]:
#devuelve si cumple el lift o el min
def bus2(buscar,Soporte,min_confidence, j,min_lift):
    if (buscar in Soporte.keys()):
        carro = bus(j[0])
        carro2 = bus(j[1])
        if (carro in Soporte.keys() and carro2 in Soporte.keys()):
            soporteunion = Soporte[buscar]
            soporteantecedente = Soporte[carro]
            soporteconsecuente = Soporte[carro2]
            if (carro2 in Soporte.keys()):
                confianza = (soporteunion / soporteantecedente)
                if ( confianza >= min_confidence):
                    lift = (soporteunion / soporteconsecuente)
                    if lift > min_lift:
                        return True
                    else: False
                else:
                    return False
            else:
                return False
        else:
            return False
    else: 
        return False

Metodo para hallar las cadenas que cumplen

In [1595]:
def cadenaparabuscar(nuevo,Soporte): #arreglo de cadenas en la posicion deseada  
    aceptado = []
    cant = 0
    for i in nuevo:
        for j in i:
            union = set(j[0]) | set(j[1])
            ver=bus(union)
            verdad = bus2(ver,Soporte,0.2,j,0.5)
            cant += 1
            if (verdad == True):
                aceptado.append(j)
    return aceptado

**impresion de las cadenas que cumple**

In [1596]:
cadenaparabuscar(nuevo,Soporte)

[[['Congratulations'], [' XO TOUR Llif3', ' goosebumps']],
 [[' XO TOUR Llif3'], ['Congratulations', ' goosebumps']],
 [[' goosebumps'], ['Congratulations', ' XO TOUR Llif3']],
 [['Congratulations'], [' XO TOUR Llif3', ' HUMBLE.']],
 [[' HUMBLE.'], ['Congratulations', ' XO TOUR Llif3']],
 [[' HUMBLE.'], ['XO TOUR Llif3', ' goosebumps']],
 [[' goosebumps'], ['XO TOUR Llif3', ' HUMBLE.']],
 [['Congratulations'], [' Bad and Boujee (feat. Lil Uzi Vert)', ' HUMBLE.']],
 [[' HUMBLE.'], ['Congratulations', ' Bad and Boujee (feat. Lil Uzi Vert)']],
 [['Congratulations'], [' Mask Off', ' HUMBLE.']],
 [[' Mask Off'], ['Congratulations', ' HUMBLE.']],
 [[' HUMBLE.'], ['Congratulations', ' Mask Off']],
 [['Bad and Boujee (feat. Lil Uzi Vert)'],
  [' Broccoli (feat. Lil Yachty)', ' Bounce Back']],
 [[' Broccoli (feat. Lil Yachty)'],
  ['Bad and Boujee (feat. Lil Uzi Vert)', ' Bounce Back']],
 [[' Bounce Back'],
  ['Bad and Boujee (feat. Lil Uzi Vert)', ' Broccoli (feat. Lil Yachty)']],
 [[' Bounce 